<a href="https://colab.research.google.com/github/agpo-ilr-uni-bonn/PromotionskollegModule6800_2024/blob/master/labIntro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Intro notebook for the course: "Machine Learning in applied economic analysis"
### Promotionskolleg Module 6800, September 2-6, 2024

#### Instructors
- Kathy Baylis
- Thomas Heckelei
- Hugo Storm

#### Description
This notebook is intended to get you familiar with some of the most common data science / ML libaries typically used in python.
In this notebook you will 1) load data, 2) prepare the data for running your models, 3) run a simple logistic regression, 4) run a very simiple neural network, and 5) compare the results of the two models. You will learn in the course that a logistic regression is actually a special case of a very simple neural network! So if you have run a logistic regression you have actually worked with NN...

Work Steps

1. (If you are reading this in Github and haven't yet opened it in colab,) Open this notebook in google colab (https://colab.research.google.com/) using the link provided above. To run the notebook you need to have a google account.

2. Execute all code cells below (Runtime/Run all) and try to understand what is going on.

3. Two important python libraries for working with data in python are numpy and pandas. There are plenty of tutorials online to get you a first idea of how they work. Two examples are provided here. For taking the course you do not have to be an expert in using those libraries but having a first basic understanding of the functionality will certainly help you to follow the examples.
    
- Numpy: https://www.datacamp.com/community/tutorials/python-numpy-tutorial

- Pandas: https://pandas.pydata.org/pandas-docs/stable/getting_started/10min.html#min   

4. (Optional) Play around with the notebook and make some changes (no worries you can not break it...). Here are some ideas what you can try to achieve:

- In the data set there are many more variables. Figure out how they are named and add a couple more variables to two models. Run the models and see how this changes the quality of the model prediction (in terms of R²).

-  If you want to go a step further... Create some new variables by adding interaction terms or square/cube terms. See if this increases model performance (R²).

- Are you up for the challenge (before even starting with the course)? The sklearn libary implements a large number of ML models. We will cover the most important ones in this course. In this notebook you have already seen how to use the logistic regression or a neural network in sklearn. Try to adjust the code to run an additional model, for example a random forest (will be covered on day 2 in the course). There are plenty of tutorials online (for example https://www.datacamp.com/community/tutorials/random-forests-classifier-python). Hint: there is basically only one line of code that you need to change in order to run an random forest with sklearn instead of a logistic regression.

#### Load relevant libs

In [3]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler
import scipy.stats as stat
from scipy.stats import norm


#### Import data

In [6]:
# run this cell only once if you don't have wget installed
# its assumed you are using windows and have python installed
# only needed if you are running the notebook locally
%pip install wget
if not os.path.isfile('brazil_all_data_v2.gz'):
    !python -m wget  https://ilr-ml.s3.eu-central-1.amazonaws.com/brazil_all_data_v2.gz
# Download data only once and make sure it is in the same folder as the notebook

# check if brazil_all_data_v2.gz is available in the current folder and if not, download it

if not os.path.isfile('brazil_all_data_v2.gz'):
    !wget  https://ilr-ml.s3.eu-central-1.amazonaws.com/brazil_all_data_v2.gz


Note: you may need to restart the kernel to use updated packages.


In [11]:
# Load data with pandas into a dataframe
df = pd.read_parquet('brazil_all_data_v2.gz')

df.head()

print(df.shape)

   id  row  col        lon        lat       bean     carrot  cassava  \
0   0    0    0 -59.989876 -10.010125  200.00000  335.00000    201.0   
1   1    0    1 -59.969875 -10.010125  200.00000  335.00000    201.0   
2   2    0    2 -59.949875 -10.010125  200.00000  335.00000    201.0   
3   3    0    3 -59.929874 -10.010125  200.00000  335.00000    201.0   
4   4    0    4 -59.909874 -10.010125  218.33334  435.83334    216.0   

   chickpea  citrus  ...  tot_defor_2010_lag_3rd_order  \
0       0.0   391.0  ...                      1.800000   
1       0.0   391.0  ...                      1.052631   
2       0.0   391.0  ...                      3.652174   
3       0.0   391.0  ...                      3.814815   
4       0.0   523.5  ...                      8.296296   

   tot_defor_2011_lag_3rd_order  tot_defor_2012_lag_3rd_order  \
0                      1.333333                      6.866667   
1                      2.000000                      5.105263   
2                      

#### Setup dependent and explantory variables

In [13]:
# Define binary variable for deforestration called D_defor_2018 from defor_2018
df['D_defor_2018'] = df['defor_2018']>0 # true false

print(df['D_defor_2018'].head(10))

0    False
1     True
2    False
3    False
4     True
5     True
6     True
7    False
8     True
9     True
Name: D_defor_2018, dtype: bool


In [14]:
# Add a variable, called constant, with only ones to the dataframe
df['constant'] = 1

In [15]:
# View first 5 rows of the data
df.head(5)

,id,row,col,lon,lat,bean,carrot,cassava,chickpea,citrus,...,tot_defor_2012_lag_3rd_order,tot_defor_2013_lag_3rd_order,tot_defor_2014_lag_3rd_order,tot_defor_2015_lag_3rd_order,tot_defor_2016_lag_3rd_order,tot_defor_2017_lag_3rd_order,tot_defor_2018_lag_3rd_order,s,D_defor_2018,constant
0,0,0,0,-59.989876,-10.010125,200.00000,335.00000,201.0,0.0,391.0,...,6.866667,0.733333,2.200000,4.466667,9.866667,6.600000,0.800000,1,False,1
1,1,0,1,-59.969875,-10.010125,200.00000,335.00000,201.0,0.0,391.0,...,5.105263,0.526316,0.947368,1.473684,9.473684,6.210527,2.000000,1,True,1
2,2,0,2,-59.949875,-10.010125,200.00000,335.00000,201.0,0.0,391.0,...,5.913043,4.086957,4.521739,4.956522,8.695652,11.217392,5.173913,1,False,1
3,3,0,3,-59.929874,-10.010125,200.00000,335.00000,201.0,0.0,391.0,...,5.407407,4.000000,3.925926,3.703704,5.888889,19.629629,6.518518,1,False,1
4,4,0,4,-59.909874,-10.010125,218.33334,435.83334,216.0,0.0,523.5,...,5.222222,7.592592,5.370370,4.481482,8.888889,18.888889,5.222222,1,True,1


In [18]:
# Define the dependent variable
Y = df['D_defor_2018'] # create a vector of just the outcomes  

# Define a list of variable names for explanatory variables
lstCols = [
  'wdpa_2017',
  'population_2015',
  'chirps_2017',
  'defor_2017',
  'maize',
  'soy',
  'sugarcane',
  'perc_treecover',
  'perm_water',
  'travel_min',
  'cropland',
  # 'pasture',
  'mean_elev',
  'sd_elev',
  'near_road',
  'defor_2017_lag_1st_order',
  'wdpa_2017_lag_1st_order',
  'chirps_2017_lag_1st_order',
  'population_2015_lag_1st_order',
  'maize_lag_1st_order',
  'soy_lag_1st_order',
  'sugarcane_lag_1st_order',
  'perc_treecover_lag_1st_order',
  'perm_water_lag_1st_order',
  'travel_min_lag_1st_order',
  'cropland_lag_1st_order',
  # 'pasture_lag_1st_order',
  'mean_elev_lag_1st_order',
  'sd_elev_lag_1st_order',
  'near_road_lag_1st_order',
#  'bean',
#  'carrot',
#  'cassava',
#  'chickpea',
#  'citrus',
#  'coffee',
#  'groundnut',
#  'maize',
#  'soy',
#  'sugarcane',
#  'tomato',
#  'wheat',
#  'defor_2001',
#  'defor_2002',
#  'defor_2003',
#  'defor_2004',
#  'defor_2005',
#  'defor_2006',
#  'defor_2007',
#  'defor_2008',
#  'defor_2009',
#  'defor_2010',
#  'defor_2011',
#  'defor_2012',
#  'defor_2013',
#  'defor_2014',
#  'defor_2015',
#  'defor_2016',
#  'defor_2017',
#  'near_dist_km',
#  'mean_elev_mts',
#  'sd_elev_mts',
 ]

# Get the explanatory Variables

# .loc is a way to select the columns 
# `:` selects all the rows
# the lstCols is the list of columns we want to select from the data 

X =  df.loc[:,lstCols]

X.head()

,wdpa_2017,population_2015,chirps_2017,defor_2017,maize,soy,sugarcane,perc_treecover,perm_water,travel_min,...,maize_lag_1st_order,soy_lag_1st_order,sugarcane_lag_1st_order,perc_treecover_lag_1st_order,perm_water_lag_1st_order,travel_min_lag_1st_order,cropland_lag_1st_order,mean_elev_lag_1st_order,sd_elev_lag_1st_order,near_road_lag_1st_order
0,1.0,0.069316,2038.0789,0.009531,461.00000,209.00000,1295.0000,99.761093,1.0,2612.6440,...,461.00000,209.00000,1295.0000,99.650558,1.0,2590.3511,0.0,238.30879,32.883953,209.46098
1,1.0,0.069481,2038.0789,0.006562,461.00000,209.00000,1295.0000,99.777657,1.0,2680.3191,...,473.98245,214.08772,1296.2280,99.635925,1.0,2634.8425,0.0,238.81532,32.887760,210.06200
2,1.0,0.069645,2037.1292,0.005313,461.00000,209.00000,1295.0000,99.766403,1.0,2796.3284,...,484.59418,218.24637,1297.2319,99.644539,1.0,2677.8906,0.0,238.43069,31.007774,210.43365
3,1.0,0.069809,2036.1794,0.000469,461.00000,209.00000,1295.0000,99.814842,1.0,2920.0164,...,492.06171,221.17284,1297.9382,99.604378,1.0,2719.8706,0.0,239.46907,32.033943,210.62392
4,1.0,0.075217,2036.1794,0.000000,522.66663,233.16666,1300.8334,99.655937,1.0,2977.4216,...,500.74072,224.57407,1298.7593,99.568802,1.0,2779.6533,0.0,246.70377,34.055607,211.30115


In [19]:
# Split the data into train and test data using sklearn train_test_split object

#   (see: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)

#   Note: This randomly splits the data in 80% train and 20% test data
X_train_raw, X_test_raw, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2)

In [23]:
X_train_raw.head(5)


,wdpa_2017,population_2015,chirps_2017,defor_2017,maize,soy,sugarcane,perc_treecover,perm_water,travel_min,...,maize_lag_1st_order,soy_lag_1st_order,sugarcane_lag_1st_order,perc_treecover_lag_1st_order,perm_water_lag_1st_order,travel_min_lag_1st_order,cropland_lag_1st_order,mean_elev_lag_1st_order,sd_elev_lag_1st_order,near_road_lag_1st_order
84215,0.0,0.312662,1994.8783,0.004531,1399.0000,876.0000,3225.0000,13.787031,1.000000,311.74936,...,1412.42990,865.31073,3245.2070,39.435654,1.000055,323.294010,0.111629,425.817570,8.508912,6.529878
238089,0.0,0.065439,1084.3052,0.000000,3680.6667,1250.3334,2695.1665,3.203906,1.000000,175.28607,...,3570.30620,1215.71640,2621.8599,4.183395,1.000000,169.653210,0.042010,83.092270,2.305456,3.514460
134121,0.0,0.836103,1612.0486,0.029687,1908.0000,928.0000,2559.0000,21.228437,1.000156,1022.87950,...,1907.59310,917.14758,2548.5867,59.743820,1.000423,968.982120,0.050072,187.344120,8.212090,27.280664
228121,0.0,0.166125,1127.9695,0.000000,3171.0000,1105.0000,3007.0000,0.020312,1.000000,60.71125,...,3205.64060,1113.96120,3015.7854,21.625488,1.033119,92.096024,0.002361,86.587601,1.625926,2.449462
22138,0.0,0.015828,1991.5323,0.000156,374.0000,192.0000,838.0000,80.983437,1.000000,744.65094,...,573.85535,308.49075,1364.2876,89.668381,1.005173,839.513000,0.000054,348.337860,27.602272,251.398510


In [24]:
# Scale data to 0-1 range using sklearn MinMaxScalar object. This facilitates training the model
# (see: https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html)
scaler = MinMaxScaler()
# Use only the train data to fit the MinMaxScalar
scaler.fit(X_train_raw)

# Apply the MinMax transformation to the train and test data
X_train = scaler.transform(X_train_raw)
X_test = scaler.transform(X_test_raw)
# Note the depended variable does not need to be scaled as it is a binary variable anyway

# don't understand why this transformation is needed

In [25]:
traindf = pd.DataFrame(X_train)
traindf.head(5)

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
0,0.0,0.000047,0.624824,0.004531,0.212786,0.426647,0.511770,0.137870,0.000000,0.081780,...,0.199981,0.398449,0.494724,0.394371,0.000068,0.087877,0.115798,0.379887,0.085298,0.015680
1,0.0,0.000010,0.131519,0.000000,0.578380,0.613440,0.426341,0.032039,0.000000,0.045866,...,0.565090,0.584035,0.388770,0.041815,0.000000,0.045230,0.043579,0.003596,0.019457,0.007139
2,0.0,0.000127,0.417425,0.029687,0.294344,0.452595,0.404386,0.212284,0.000156,0.268932,...,0.283762,0.425903,0.376315,0.597472,0.000522,0.267105,0.051942,0.118058,0.082148,0.074452
3,0.0,0.000025,0.155174,0.000000,0.496715,0.540918,0.476620,0.000203,0.000000,0.015713,...,0.503389,0.530142,0.455728,0.216252,0.040832,0.023701,0.002449,0.007434,0.012245,0.004123
4,0.0,0.000002,0.623011,0.000156,0.048550,0.085329,0.126895,0.809834,0.000000,0.195709,...,0.058095,0.103538,0.175014,0.896746,0.006377,0.231168,0.000056,0.294819,0.287948,0.709223


In [26]:
# Fit a logistic regression model using sklearn (see: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)
# Create the model object
modelLg = LogisticRegression(random_state=0,penalty=None,fit_intercept=True, max_iter=1000)
# Fit the model using the training data
modelLg.fit(X_train, Y_train)

LogisticRegression(max_iter=1000, penalty=None, random_state=0)

### Note:
sklearn is a popular ML libary that we will primarily use in the course. While sklearn allows to run
regressions it does not provide regression table outputs (with p-values, standard errors etc.).
While those table are very common in econometrics they are not commonly considered in the ML
community. For illustrative puposes we do the calculation for a regression table manually, however,
there is also a "statsmodels" libary in python that does this automatically (see below).

In [27]:
# Function to calculate pvalues and standard errors for a scikit-learn logisticRegression
# Source: https://stackoverflow.com/questions/25122999/scikit-learn-how-to-check-coefficients-significance
def logit_pvalue(model, x):
    """ Calculate z-scores for scikit-learn LogisticRegression.
    parameters:
        model: fitted sklearn.linear_model.LogisticRegression with intercept and large C
        x:     matrix on which the model was fit
    This function uses asymtptics for maximum likelihood estimates.
    """
    p = model.predict_proba(x)
    n = len(p)
    m = len(model.coef_[0]) + 1
    # m = len(model.coef_[0])
    # coefs = model.coef_[0]
    coefs = np.concatenate([model.intercept_, model.coef_[0]])
    x_full = np.matrix(np.insert(np.array(x), 0, 1, axis = 1))
    ans = np.zeros((m, m))
    for i in range(n):
        ans = ans + np.dot(np.transpose(x_full[i, :]), x_full[i, :]) * p[i,1] * p[i, 0]
    vcov = np.linalg.inv(np.matrix(ans))
    se = np.sqrt(np.diag(vcov))
    t =  coefs/se
    p = (1 - norm.cdf(abs(t))) * 2
    return se, p

In [28]:
# Use the previously created function to create a regression output table
se, p = logit_pvalue(modelLg, X_train)
coefs = np.concatenate([modelLg.intercept_, modelLg.coef_[0]]).T
resCoef = pd.DataFrame(coefs,index=['constant']+lstCols)
resCoef.columns = ['coef']
resCoef['se'] = se
resCoef['pval'] = p
resCoef

,coef,se,pval
constant,-1.938529,0.034284,0.000000e+00
wdpa_2017,-0.473744,0.064229,1.632028e-13
population_2015,0.789097,0.488295,1.060891e-01
chirps_2017,1.274356,0.690596,6.499398e-02
defor_2017,14.286842,0.288499,0.000000e+00
maize,0.404382,0.769550,5.992509e-01
soy,1.452906,0.575346,1.156087e-02
sugarcane,-1.324143,0.509393,9.337285e-03
perc_treecover,0.686018,0.032792,0.000000e+00
perm_water,-0.378337,0.275854,1.702165e-01


In [30]:
# Confirm the results using statsmodels
import statsmodels.api as sm
# Add constant to X matrix
X_train_const = np.matrix(np.insert(np.array(X_train), 0, 1, axis = 1))

# Define the logit regression
logit = sm.Logit(Y_train,X_train_const)

# Set the names of the explanatory variables
logit.data.xnames = exog_names=['const']+lstCols

# fit the model
result = logit.fit()
# Print the summary table
print(result.summary())

Optimization terminated successfully.
         Current function value: 0.469925
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:           D_defor_2018   No. Observations:               199952
Model:                          Logit   Df Residuals:                   199923
Method:                           MLE   Df Model:                           28
Date:                Mon, 30 Sep 2024   Pseudo R-squ.:                  0.1457
Time:                        13:49:48   Log-Likelihood:                -93963.
converged:                       True   LL-Null:                   -1.0999e+05
Covariance Type:            nonrobust   LLR p-value:                     0.000
                                    coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
const                            -1.9166      0.034    -55.839      0.

## Train your first very (very) simple neural network using sklearn
Now use a neural network for the same problem. In the course you will see that this is actually equivalent to a logistic regression, hence a logistic regression is in fact a specific form of a neural network!

### Perform a hyper parameter search to tune the learning rate for training the NN.
This step is optional and takes a while. You can also run the next cell,
using a fixed learning rate. The learning rate was determined using this hyper parameter search.

In [ ]:
from scipy.stats import loguniform
from sklearn.model_selection import RandomizedSearchCV
param_grid = {
    'alpha':loguniform(1e-6, 1e-1)}

modelNN = MLPClassifier(solver='lbfgs', activation = 'identity',
                     hidden_layer_sizes=(1), random_state=1, verbose=True,max_iter=200)


clf = RandomizedSearchCV(modelNN, param_grid, random_state=0,n_iter=10,cv=5)

X_train_const_ = np.asarray(X_train_const)
Y_train_ = np.asarray(Y_train)
modelNN = clf.fit(X_train_const_, Y_train_)
modelNN.best_params_

### Train the Neural Network with a fixed set of hyperparameter

In [ ]:
modelNN = MLPClassifier(solver='lbfgs', alpha=8.264328927007723e-05,activation = 'identity',
                     hidden_layer_sizes=(1), random_state=1, verbose=True,max_iter=200)

modelNN.fit(X_train_const_, Y_train_)

In [ ]:
# Add the estimated coefficient of the NN to the regression table we created above-
# In the course we will discuss why the estimated coefficient are similar.
#    modelNN.coefs_[0] are the coefficients of the first layer
#    modelNN.coefs_[1][0][0] is the coefficients of the hidden layer
resCoef['coef_NN'] = modelNN.coefs_[0]*modelNN.coefs_[1][0][0]
resCoef

### Compare the model outcomes

In [ ]:
# Add constant to the test data
X_test_const = np.asarray(np.insert(np.array(X_test), 0, 1, axis = 1))
# Get predicted values from logit model
Y_test_Lg = modelLg.predict(X_test)
# Get predicted values from NN model
Y_test_NN = modelNN.predict(X_test_const)

In [ ]:
score_Lg = np.sum(Y_test==Y_test_Lg)/Y_test.shape[0]
score_NN = np.sum(Y_test==Y_test_NN)/Y_test.shape[0]
print('Score lg (R²): ',score_Lg)
print('Score NN (R²): ',score_NN)

In [ ]:
# plot the predicted probabalities of the logit and NN models
fig, (ax1, ax2) = plt.subplots(1, 2, sharey=True)

pd.DataFrame(modelLg.predict_proba(X_test))[1].hist(bins=100,ax=ax1)
pd.DataFrame(modelNN.predict_proba(X_test_const))[1].hist(bins=100,ax=ax2)
fig.show()

### Well done!!!
Now it is your turn. Play around with the notebook to make your very first steps with numpy/pandas and sklearn. In the intro text in the beginning there are some suggestions of what you can try.  